In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [1]:
!ls

drive  sample_data


In [2]:
cd /content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet


In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 8.1 MB/s 
     |████████████████████████████████| 180 kB 75.4 MB/s 
     |████████████████████████████████| 143 kB 71.3 MB/s 
     |████████████████████████████████| 97 kB 9.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=b46d894ea53430949826f9a4f08dd6c3e12a269eca79253e68c96d6025912633
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d8e5c233bef1c26f9ad23039178641a93580200c43f0eeaa3ded77481e615018
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Some hyperparameters and settings

In [6]:
CONFIG_FILE_PATH = 'config/inD_longterm.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'ind_longterm'  # arbitrary name for this experiment
DATASET_NAME = 'ind'

TRAIN_DATA_PATH = 'data/inD/train.pkl'
TRAIN_IMAGE_PATH = 'data/inD/train'
VAL_DATA_PATH = 'data/inD/test.pkl'
VAL_IMAGE_PATH = 'data/inD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 5  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 8

In [11]:
pip install -r requirements.txt

     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 269 kB 18.0 MB/s 
     |████████████████████████████████| 753.2 MB 13 kB/s 
     |████████████████████████████████| 49.4 MB 288 kB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 58 kB 7.9 MB/s 
     |████████████████████████████████| 23.3 MB 98.8 MB/s 
     |████████████████████████████████| 23.3 MB 109 kB/s 
     |████████████████████████████████| 22.1 MB 27.2 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 17.4 MB 61.1 MB/s 
     |████████████████████████████████| 17.3 MB 40.6 MB/s 
     |████████████████████████████████| 12.8 MB 71.4 MB/s 
     |████████████████████████████████| 12.7 MB 44.0 MB/s 
     |████████████████████████████████| 11.8 MB 82.3 MB/s 
     |████████████████████████████████| 5.9 MB 72.0 MB/s 
     |███████

#### Load config file and print hyperparameters

In [7]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.002,
 'resize': 0.33,
 'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.8,
 'unfreeze': 100,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [14, 29]}

#### Wandb INIT

#### Load preprocessed Data

In [8]:
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

In [9]:
df_train.head()

,trackId,frame,x,y,sceneId,metaId
0,31,2217,25.07654,6.78323,07,0
1,31,2242,26.11484,7.72170,07,0
2,31,2267,27.05390,8.94723,07,0
3,31,2292,28.08326,10.18219,07,0
4,31,2317,29.08530,11.39276,07,0


#### Initiate model

In [10]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
import weights_and_biases as wandb
wandb.init_wandb(params.copy(), model.model)

wandb: Currently logged in as: agv (use `wandb login --relogin` to force relogin)


In [25]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH, 
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name= 'ind')

KeyboardInterrupt: ignored